# Lademanagementsimulation   


### Simulations Parameter

In [1]:
solarpeakleistung = 125 #Einheit Kilowattpeak (kWP)
anzahl_bevs_pro_tag = 1
ladeleistung_pro_bev = 11 #Einheit kW
maximale_ladezeit = 30 #Einheit Minuten
minute_interval = 15 #Einheit Minuten

### Simulations Eingangsdaten

In [2]:
import simulationInputDataVisualisation
simulationInputDataVisualisation.create_jupyter_dash_app(solarpeakleistung)

### Verteilungsalgorithmus

In [3]:
import simulationClasses
import simulationData
import simulationOutputDataVisualisation
import distributionAlgorithmForecastPolling
import postOptimization
import numpy as np

simulation_day = simulationClasses.SimulationDay(anzahl_bevs_pro_tag)
bev_data = simulationData.BevData()
table_dict = simulationData.TableDict()
simulation_data = simulationData.SimulationData()
    
day_in_minute_interval_steps = list(np.around(np.arange(480, 960 + 1, minute_interval), 1))
distributionAlgorithmForecastPolling.init_simulation(day_in_minute_interval_steps, minute_interval, simulation_data, 
                                                   simulation_day, solarpeakleistung)

postOptimization.start_post_optimization(minute_interval, simulation_day, solarpeakleistung, bev_data, table_dict, simulation_data,
                ladeleistung_pro_bev)

distributionAlgorithmForecastPolling.start_simulation(solarpeakleistung, ladeleistung_pro_bev,
                     simulation_day, bev_data, table_dict, simulation_data, minute_interval)

START POST OPTIMIZATION


OPTIMIZATION DONE
BEVs dict:  {0: [(13.0, 3.0), 'wartend', [(951.8774703557312, 8.122529644268752, 0)], [1.37]], 1: [(9.0, 3.4), 'nicht parkend', [], [2.2424999999999997]], 2: [(8.0, 4.1), 'nicht parkend', [], [2.3074999999999997]], 3: [(8.5, 5.2), 'nicht parkend', [], [3.0725000000000002]], 4: [(12.0, 4.0), 'wartend', [(936.8578352180938, 15.019635137637465, 0)], [2.0]], 5: [(8.0, 6.3), 'nicht parkend', [], [3.6374999999999993]], 6: [(8.5, 6.5), 'wartend', [(699.1773343373494, 20.822665662650593, 0)], [3.7500000000000004]], 7: [(8.0, 4.5), 'wartend', [(689.1713231305645, 9.878940423652352, 0)], [2.4774999999999996]], 8: [(9.0, 3.2), 'nicht parkend', [], [2.0725]], 9: [(8.0, 4.2), 'nicht parkend', [], [2.3074999999999997]], 10: [(12.5, 3.3), 'nicht parkend', [], [1.6625]], 11: [(8.0, 6.1), 'nicht parkend', [], [3.4924999999999993]], 12: [(13.5, 2.5), 'wartend', [(934.2407108239096, 10.75928917609042, 0)], [1.1099999999999999]], 13: [(8.0, 3.0), 'wartend', [(65

SET BEV DATA charging start between intervals for ID BEV 9 for minute 517.3920863309353:  5.56
AVAILABLE SOLAR POWER:  11.12
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 5:  3.6374999999999993
Restladezeit:  39.25359712230215
Charging BEVs:  [2, 5]


Minute:  525
Waiting BEVs:  [11, 18, 41, 50, 59, 64, 70, 80, 81, 85, 3, 17, 19, 20, 21, 22, 25, 26, 27, 28, 33, 34, 36, 38, 45, 48, 51, 53, 57, 60, 73, 77, 91, 92, 93, 95, 99]
Waiting BEVs after updating parking end:  [11, 18, 41, 50, 59, 64, 70, 80, 81, 85, 3, 17, 19, 20, 21, 22, 25, 26, 27, 28, 33, 34, 36, 38, 45, 48, 51, 53, 57, 60, 73, 77, 91, 92, 93, 95, 99]
Charging BEVs after updating post optimization plan and parking end:  [5, 9]
FUEL Charging BEV:  5
SET BEV DATA for current minute for ID BEV 5 for minute 525:  8.19
SET BEV DATA for current minute for ID BEV 9 for minute 525:  8.19
Charging BEVs (nachdem neue Plätze belegt):  [5, 9]
AVAILABLE SOLAR POWER:  16.38
Restladeenergie bis zur fairen Ladeenergie for BEV mit 

SET BEV DATA for current minute for ID BEV 85 for minute 600:  10.176
Charging BEVs (nachdem neue Plätze belegt):  [41, 70, 80, 81, 85]
AVAILABLE SOLAR POWER:  50.88
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 41:  -7.489999999999997
Restladezeit:  -44.16273584905658
AVAILABLE SOLAR POWER:  50.88
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 70:  1.165000000000001
Restladezeit:  6.869103773584911
Die Restladezeit des BEVS 70 endet vor dem nächsten Interval
SET BEV DATA after charging time over for ID BEV 70 for minute 606.8691037735849:  10.178
ALLOCATE FREED SOLAR ENERGY ;)
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  3
Ladestart für ausgewähltes BEV:  606.8691037735849
SET BEV DATA for charging start of new bev for ID BEV 3 for minute 606.8691037735849:  10.176
SET BEV DATA charging start between intervals for ID BEV 3 for minute 606.8691037735849:  10.176
AVAILABLE SOLAR POWER:  50.88
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 80

Minute:  675
Waiting BEVs:  [34, 36, 38, 45, 48, 51, 53, 57, 60, 73, 77, 91, 92, 93, 95, 99, 1, 8, 15, 24, 37, 43, 46, 55, 68, 72, 87, 39, 65, 96, 58]
SET BEV DATA charging start between intervals for ID BEV 7 for minute 689.1713231305645:  8.646666666666667
SET BEV DATA charging start between intervals for ID BEV 35 for minute 683.37847471008:  8.646666666666667
SET BEV DATA charging start between intervals for ID BEV 86 for minute 689.8812418917205:  8.646666666666667
Waiting BEVs after updating parking end:  [34, 36, 38, 45, 48, 51, 53, 57, 60, 73, 77, 91, 92, 93, 95, 99, 1, 8, 15, 24, 37, 43, 46, 55, 68, 72, 87, 39, 65, 96, 58]
Charging BEVs after updating post optimization plan and parking end:  [26, 28, 33]
FUEL Charging BEV:  26
FUEL Charging BEV:  28
SET BEV DATA for current minute for ID BEV 26 for minute 675:  17.293333333333333
SET BEV DATA for current minute for ID BEV 28 for minute 675:  17.293333333333333
SET BEV DATA for current minute for ID BEV 33 for minute 675:  17.2

Minute:  750
Waiting BEVs:  [34, 36, 38, 45, 48, 51, 53, 57, 60, 73, 77, 91, 92, 93, 95, 99, 1, 24, 37, 43, 72, 87, 39, 65, 96, 58, 79, 94, 97]
SET BEV DATA charging start between intervals for ID BEV 74 for minute 762.9713248935939:  7.1866666666666665
Waiting BEVs after updating parking end:  [34, 36, 38, 45, 48, 51, 53, 57, 60, 73, 77, 91, 92, 93, 95, 99, 24, 37, 43, 72, 87, 39, 65, 96, 58, 79, 94, 97, 10, 75]
Charging BEVs after updating post optimization plan and parking end:  [78, 82, 84, 89, 62]
FUEL Charging BEV:  78
FUEL Charging BEV:  82
FUEL Charging BEV:  84
FUEL Charging BEV:  89
FUEL Charging BEV:  62
SET BEV DATA for current minute for ID BEV 78 for minute 750:  8.623999999999999
SET BEV DATA for current minute for ID BEV 82 for minute 750:  8.623999999999999
SET BEV DATA for current minute for ID BEV 84 for minute 750:  8.623999999999999
SET BEV DATA for current minute for ID BEV 89 for minute 750:  8.623999999999999
SET BEV DATA for current minute for ID BEV 62 for min



Minute:  825
Waiting BEVs:  [36, 38, 48, 53, 73, 77, 91, 92, 93, 95, 99, 24, 43, 72, 87, 96, 79, 94, 97, 10, 75, 71, 83, 88, 90]
SET BEV DATA charging start between intervals for ID BEV 56 for minute 830.0945454545455:  8.25
SET BEV DATA after charging time over for ID BEV 63 for minute 830.0945454545455:  1.7830434782608695
Waiting BEVs after updating parking end:  [36, 38, 48, 53, 73, 77, 91, 92, 95, 99, 24, 43, 72, 87, 96, 79, 94, 97, 10, 75, 71, 83, 88, 90]
Charging BEVs after updating post optimization plan and parking end:  [62, 44, 52, 63]
FUEL Charging BEV:  62
FUEL Charging BEV:  44
FUEL Charging BEV:  52
SET BEV DATA for current minute for ID BEV 62 for minute 825:  10.3125
SET BEV DATA for current minute for ID BEV 44 for minute 825:  10.3125
SET BEV DATA for current minute for ID BEV 52 for minute 825:  10.3125
SET BEV DATA for current minute for ID BEV 63 for minute 825:  10.3125
Charging BEVs (nachdem neue Plätze belegt):  [62, 44, 52, 63]
AVAILABLE SOLAR POWER:  41.25


SET BEV DATA for current minute for ID BEV 29 for minute 900:  2.813333333333333
SET BEV DATA for current minute for ID BEV 31 for minute 900:  2.813333333333333
Charging BEVs (nachdem neue Plätze belegt):  [62, 44, 52, 32, 29, 31]
AVAILABLE SOLAR POWER:  16.88
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 62:  -23.145375
Restladezeit:  -493.6217417061612
AVAILABLE SOLAR POWER:  16.88
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 44:  -18.457875
Restladezeit:  -393.6513625592418
AVAILABLE SOLAR POWER:  16.88
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 52:  -18.457875
Restladezeit:  -393.6513625592418
AVAILABLE SOLAR POWER:  16.88
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 32:  1.0611932214746571
Restladezeit:  22.632082922445296
AVAILABLE SOLAR POWER:  16.88
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 29:  3.5562722298221683
Restladezeit:  75.84466840852966
AVAILABLE SOLAR POWER:  16.88
Restladeenergie bis zur fairen La

### Ergebnis Algorithmus

In [4]:
simulationOutputDataVisualisation.print_algorithm_result(bev_data, simulation_data, anzahl_bevs_pro_tag, solarpeakleistung, minute_interval)

>**Ergebnis**<br>Geladene BEVs: 60 von 100<br>Verfügbare Solarenergie insgesamt: 276.47 kWh<br>Aufgeladene Solarenergie insgesamt: 232.57 kWh<br>Ungenutzte Solarenergie insgesamt: 0 kWh

### Tabellarische Übersicht Park- und Ladeverlauf der BEVs

In [5]:
simulationOutputDataVisualisation.create_tabular_overview_per_minute_slider(table_dict, minute_interval)

interactive(children=(IntSlider(value=480, description='Minute: ', max=960, min=480, step=15), Output()), _dom…

In [6]:
import figureGeneration

figureGeneration.create_bev_number_figure(simulation_data)

In [7]:
figureGeneration.create_charging_power_figure(simulation_day, solarpeakleistung, bev_data, minute_interval)

{2: {480: 0.62, 495: 5.87, 510: 5.56, 517.3920863309353: 5.545}, 5: {510: 5.56, 525: 8.19, 540: 7.666666666666667, 541.5652173913044: 6.609999999999999}, 9: {517.3920863309353: 5.56, 525: 8.19, 536.7399267399268: 8.93}, 11: {536.7399267399268: 8.19, 540: 7.666666666666667, 555: 10.333333333333334, 561.5661290322581: 10.166666666666666}, 18: {540: 7.666666666666667, 555: 10.333333333333334, 559.141935483871: 9.736666666666666}, 41: {541.5652173913044: 7.666666666666667, 555: 10.333333333333334, 570: 9.75, 585: 9.4, 600: 10.176, 615: 10.123999999999999, 630: 10.076, 645: 12.53, 660: 10.123999999999999, 672: 8.5}, 50: {561.5661290322581: 10.333333333333334, 570: 9.75, 581.5538461538462: 10.29}, 59: {570: 9.75, 585: 9.4, 587.1223404255319: 8.826}, 64: {570: 9.75, 585: 9.4, 595.8670212765958: 9.76}, 70: {581.5538461538462: 9.75, 585: 9.4, 600: 10.176, 606.8691037735849: 10.178}, 80: {585: 9.4, 600: 10.176, 610.1709905660377: 10.166}, 81: {587.1223404255319: 9.4, 600: 10.176, 600.72228773584